# Quantum Process Tomography via Neural Networks
## SU(2) Transformations
___

This notebook performs the process tomography for the numerical experiments proposed in our paper 'Retrieving unitary polarization transformations via optimized quantum tomography'.
 

   The notebook is organized as follows: 

   1. Importing the synthetic data
   2. Importing the network and performing the process tomography with 6 measurements
   3. Evaluating the fidelity of the reconstruction for each process

First, the required libraries are imported:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GaussianDropout
from tensorflow.keras import optimizers
import tensorflow as tf

By running the following cell, you import the experimental data.

In [ ]:
error = "0"
path = "dataset/"+ error

LL = np.loadtxt(path + "/LL.txt", dtype="f", delimiter="\t")
HH = np.loadtxt(path + "/HH.txt", dtype="f", delimiter="\t")
HL = np.loadtxt(path + "/HL.txt", dtype="f", delimiter="\t")
LD = np.loadtxt(path + "/LD.txt", dtype="f", delimiter="\t")
LH = np.loadtxt(path + "/LH.txt", dtype="f", delimiter="\t")
HD = np.loadtxt(path + "/HD.txt", dtype="f", delimiter="\t")

If available, you can import the theoretical $U_{Th}$ to compute the fidelity of the reconstructed process.

In [ ]:
path = "dataset"
theta_th = np.loadtxt(path + "/random_Theta.txt", dtype="f", delimiter="\t")
nx_th = np.loadtxt(path + "/random_nx.txt", dtype="f", delimiter="\t")
ny_th = np.loadtxt(path + "/random_ny.txt", dtype="f", delimiter="\t")
nz_th = np.loadtxt(path + "/random_nz.txt", dtype="f", delimiter="\t")

The function `compute_unitary` is used to compute the unitary $U$, given the parameters $\Theta\in[0,\pi]$ and $\mathbf{n}=(n_x,n_y,n_z)$ according to:

\begin{equation}
U=\begin{pmatrix}
\cos \Theta -i \sin \Theta \,n_z && -i\sin \Theta \,(n_x-i n_y)\\
-i\sin \Theta \,(n_x+i n_y) && \cos \Theta + i \sin \Theta \,n_z
\end{pmatrix}
\end{equation}

In [ ]:
def compute_unitary(Theta, nx, ny, nz):
    I = np.array([[1, 0], [0, 1]])
    sx = np.matrix([[0, 1], [1, 0]])
    sy = np.matrix([[0, -1j], [1j, 0]])
    sz = np.matrix([[1, 0], [0, -1]])
    return np.cos(Theta) * I - 1j * np.sin(Theta) * (nx * sx + ny * sy + nz * sz)

The function `fidelity` is used to compute the function used to measure the "distance" between the reconstructed and theoretical unitaries:

\begin{equation}
F=\frac{1}{2}\,\biggl|Tr(U_\text{th}^{\dagger}U_\text{exp})\biggr|
\end{equation}

In [ ]:
def fidelity(mat1,mat2):
    prod=np.trace(np.dot(np.conjugate(mat1.T),mat2))
    
    return 0.5*np.abs(prod)

Finally, we set the total number of evolutions to be processed

In [ ]:
#number of evolutions
num_unit=1000

_____

### Neural Network Reconstruction with 6 measurements

The following cell imports the network trained to reconstruct the evolutions with 6 inputs. 
The set of measurements is  $[LL, LH, LD, HL, HH, HD]$

In [ ]:
json_file = open(r'./models/NN_6in.json', 'r') #path of NN 6 inputs json file
loaded_model_json = json_file.read()
json_file.close()
loaded_model6 = tf.keras.models.model_from_json(loaded_model_json)
loaded_model6.load_weights(r'./models/NN_6in.h5') #path of NN 6 inputs h5 file

data6=np.zeros([num_unit,6])
data6[:,0]=LL
data6[:,1]=LH
data6[:,2]=LD
data6[:,3]=HL
data6[:,4]=HH
data6[:,5]=HD

We proceed with the network prediction:

In [ ]:
y_pred6=loaded_model6.predict(data6)
theta_vect6=y_pred6[:,0]*np.pi
nx_vect6=y_pred6[:,1]*2 -1 
ny_vect6=(y_pred6[:,2]*2 -1)*np.sqrt(1-nx_vect6**2)

nz_vect6=np.sqrt(abs(1-nx_vect6**2-ny_vect6**2))

The fidelities of individual reconstructions are calculated and plotted:

In [ ]:
Fvals6=np.zeros(num_unit)

for i in range(num_unit):
    netU=compute_unitary(theta_vect6[i],nx_vect6[i],ny_vect6[i],nz_vect6[i])
    thU=compute_unitary(theta_th[i],nx_th[i],ny_th[i],nz_th[i])
    Fvals6[i]=fidelity(netU,thU)

plt.plot(range(num_unit), Fvals6)
plt.show()

Average fidelity and standard deviation:

In [ ]:
np.mean(Fvals6), np.sqrt(np.var(Fvals6))